In [97]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
pd.options.display.float_format = '{:.2f}'.format

In [98]:
!ls -ltr

total 88
drwxr-xr-x  6 prashau  ANT\Domain Users    204 Mar  5 07:48 data
-rw-r--r--  1 prashau  ANT\Domain Users  43470 Mar  6 13:35 EDA.ipynb


In [99]:
!pwd

/Users/prashau/kaggle/walmart


In [100]:
df_old = pd.read_csv("data/old/pos_data.csv",low_memory=False,parse_dates=["visit_date"])

In [101]:
df_new = pd.read_csv("data/new/pos_data.csv",low_memory=False,parse_dates=["visit_date"])

In [286]:
add_ups_df = pd.read_csv("data/addList.csv")
delete_ups_df = pd.read_csv("data/deleteList.csv")
add_ups_df.rename(columns={'Addition_list':'upc'},inplace=True)
delete_ups_df.rename(columns={'Delete_List':'upc'},inplace=True)

In [287]:
df_old["category"] = 1
df_new["category"] = 2

In [288]:
data_array = []
data_array.append(df_old)
data_array.append(df_new)

In [289]:
df_mpos_data = pd.concat(data_array)
df_mpos_data.columns
df_mpos_data.dept_subcategory_nbr.unique().size

df_mpos_data.columns

Index(['store_nbr', 'visit_nbr', 'visit_date', 'wm_year_wk_nbr', 'upc_nbr',
       'scan_id', 'rollup_id', 'fineline_nbr', 'dept_subcategory_nbr',
       'household_id', 'brand_name', 'unit_qty', 'retail_price', 'category'],
      dtype='object')

In [290]:
df_mpos_data.sort_values("wm_year_wk_nbr",ascending=True,inplace=True)

In [291]:
print(df_mpos_data[["upc_nbr","dept_subcategory_nbr","brand_name"]].count())
df_mpos_data[["upc_nbr","dept_subcategory_nbr","brand_name"]].head(5)

upc_nbr                 3675625
dept_subcategory_nbr    3675625
brand_name              3675625
dtype: int64


,upc_nbr,dept_subcategory_nbr,brand_name
793712,2122,sub_A,BRAND_X
719660,2131,sub_A,BRAND_X
719661,2131,sub_A,BRAND_X
719662,2131,sub_A,BRAND_X
719663,2131,sub_A,BRAND_X


In [292]:
print(add_ups_df.count())
add_ups_df.head()

upc              91
Mod Drop Week    91
category         91
dtype: int64


,upc,Mod Drop Week,category
0,100462,201717,2
1,100463,201717,2
2,100464,201717,2
3,100465,201717,2
4,100466,201717,2


In [293]:
print(delete_ups_df.count())
delete_ups_df.head()

upc              22
Mod Drop Week    22
category         22
dtype: int64


,upc,Mod Drop Week,category
0,100049,201717,2
1,100057,201717,2
2,100098,201717,2
3,100113,201717,2
4,100114,201717,2


In [296]:
add_merge = add_ups_df.merge(df_mpos_data[["upc_nbr","dept_subcategory_nbr","fineline_nbr","rollup_id"]].drop_duplicates(),left_on="upc",right_on="upc_nbr")


In [297]:
delete_merge = delete_ups_df.merge(df_mpos_data[["upc_nbr","dept_subcategory_nbr","fineline_nbr","rollup_id"]].drop_duplicates(),left_on="upc",right_on="upc_nbr")


In [298]:
print(add_merge.count())
add_merge.head(5)
np.sort(add_merge.dept_subcategory_nbr.unique())


upc                     91
Mod Drop Week           91
category                91
upc_nbr                 91
dept_subcategory_nbr    91
fineline_nbr            91
rollup_id               91
dtype: int64


array(['subcat_4', 'subcat_6'], dtype=object)

In [299]:
print(delete_merge.count())
delete_merge.head(5)
np.sort(delete_merge.dept_subcategory_nbr.unique()).size


upc                     22
Mod Drop Week           22
category                22
upc_nbr                 22
dept_subcategory_nbr    22
fineline_nbr            22
rollup_id               22
dtype: int64


3

In [300]:
add_merge.groupby(["category","dept_subcategory_nbr","fineline_nbr"]).count()

upc  Mod Drop Week  upc_nbr  \
category dept_subcategory_nbr fineline_nbr                                
2        subcat_4             fl_ 17         23             23       23   
                              fl_ 18         33             33       33   
                              fl_ 19         15             15       15   
                              fl_ 20          3              3        3   
                              fl_ 21         15             15       15   
         subcat_6             fl_ 14          2              2        2   

                                            rollup_id  
category dept_subcategory_nbr fineline_nbr             
2        subcat_4             fl_ 17               23  
                              fl_ 18               33  
                              fl_ 19               15  
                              fl_ 20                3  
                              fl_ 21               15  
         subcat_6             fl_ 14                2

In [301]:
delete_merge.groupby(["category","dept_subcategory_nbr","fineline_nbr"]).count()

upc  Mod Drop Week  upc_nbr  \
category dept_subcategory_nbr fineline_nbr                                
2        subcat_4             fl_ 19          3              3        3   
         subcat_6             fl_ 11          1              1        1   
                              fl_ 12          1              1        1   
                              fl_ 13          1              1        1   
                              fl_ 14          1              1        1   
                              fl_ 4           3              3        3   
                              fl_ 8           6              6        6   
                              fl_ 9           2              2        2   
         subcat_8             fl_ 6           4              4        4   

                                            rollup_id  
category dept_subcategory_nbr fineline_nbr             
2        subcat_4             fl_ 19                3  
         subcat_6             fl_ 11                1  
                              fl_ 12                1  
                              fl_ 13                1  
                              fl_ 14                1  
                              fl_ 4                 3  
                              fl_ 8                 6  
                              fl_ 9                 2  
         subcat_8             fl_ 6                 4

In [325]:
df_add_delete_concat = pd.concat([add_ups_df,delete_ups_df])
df_add_delete_concat_merge = df_add_delete_concat.merge(df_mpos_data[["upc_nbr","dept_subcategory_nbr","fineline_nbr","rollup_id"]].drop_duplicates(),left_on="upc",right_on="upc_nbr")
df_add_delete_concat_merge.dept_subcategory_nbr.unique()
#df_add_delete_concat_merge


array(['subcat_4', 'subcat_6', 'subcat_8'], dtype=object)

## Track households from provided SKUs

In [119]:
print('##########-----CATEGORY 1-------##########')
print(f'Unique household in category 1 are {df_old["household_id"].unique().size}')
print(f'{df_old["household_id"].isnull().sum()} null householdId out of {len(df_old)}. {(df_old["household_id"].isnull().sum()/len(df_old))*100} % are null.')


print('\n##########-----CATEGORY 2-------##########')
print(f'Unique household in category 2 are {df_new["household_id"].unique().size}')
print(f'{df_new["household_id"].isnull().sum()} null householdId out of {len(df_new)}. {(df_new["household_id"].isnull().sum()/len(df_new))*100} % are null.')

print('\n##########-----CATEGORY All-------##########')
print(f'Unique household in All categories are {df_mpos_data["household_id"].unique().size}')
print(f'{df_mpos_data["household_id"].isnull().sum()} null householdId out of {len(df_mpos_data)}. {(df_mpos_data["household_id"].isnull().sum()/len(df_mpos_data))*100} % are null.')




#df_new['household_id'].isnull().count()

##########-----CATEGORY 1-------##########
Unique household in category 1 are 195773
452016 null householdId out of 839214. 53.861827853205504 % are null.

##########-----CATEGORY 2-------##########
Unique household in category 2 are 91291
2295119 null householdId out of 2836411. 80.91630585271317 % are null.

##########-----CATEGORY All-------##########
Unique household in All categories are 287063
2747135 null householdId out of 3675625. 74.73926203026696 % are null.


In [120]:
df_non_null_household_mpos_data = df_mpos_data.dropna(subset=["household_id"])

### find the rows from data which have householdId and have upc_numbers as provided in list.

In [206]:
add_merge_non_null = add_ups_df.merge(df_non_null_household_mpos_data[["upc_nbr","dept_subcategory_nbr","fineline_nbr","brand_name","household_id"]].drop_duplicates(),left_on="upc_add",right_on="upc_nbr")
delete_merge_non_null = delete_ups_df.merge(df_non_null_household_mpos_data[["upc_nbr","dept_subcategory_nbr","fineline_nbr","brand_name","household_id"]].drop_duplicates(),left_on="upc_delete",right_on="upc_nbr")

In [207]:
household_id_from_add = add_merge_non_null.household_id.unique()
household_id_from_delete = delete_merge_non_null.household_id.unique()

In [208]:
target_household_ids = np.union1d(household_id_from_add,household_id_from_delete)
print(f'Unique target households count {target_household_ids.size}')

Unique target households count 6238


In [209]:
df_non_null_household_mpos_target_data = df_non_null_household_mpos_data[df_non_null_household_mpos_data["household_id"].isin(target_household_ids)]

In [210]:
len(df_non_null_household_mpos_target_data)

96974

In [239]:
transaction_count_per_household = df_non_null_household_mpos_target_data.groupby("household_id").size().sort_values()
pd.DataFrame(transaction_count_per_household,columns=["transactions_done"]).groupby("transactions_done").size()

transactions_done
1       610
2       598
3       547
4       428
5       442
6       371
7       336
8       278
9       228
10      200
11      165
12      141
13      121
14      127
15      116
16      104
17       86
18       87
19       80
20       83
21       59
22       51
23       58
24       39
25       50
26       45
27       46
28       45
29       34
30       36
       ... 
243       1
244       1
247       1
252       1
255       1
257       2
266       1
274       1
275       1
297       1
304       1
311       1
312       1
315       1
320       1
324       1
344       1
347       1
352       1
399       1
434       1
435       1
472       1
502       1
503       1
600       1
672       1
772       1
903       1
2095      1
Length: 186, dtype: int64

,transactions_done
household_id,
hh_49882,1
hh_16884,1
hh_38857,1
hh_3877,1
hh_38729,1
hh_17003,1
hh_72133,1
hh_86113,1
hh_38665,1
